# General settings

Import anything that can be useful

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import random
import bisect
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

We read the dataset and fix reasonable parameters

In [2]:
data = pd.read_excel('data.xlsx')
data = data.dropna(axis=0)
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
GENES = X.shape[1]

# Binary genome
bases = [False, True]
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.7

GENERATIONS = 20
POPULATION_SIZE = 100
SEED = 1

Implement statistic functions

In [4]:
def mask_X(X, mask):
    return X.iloc[:, mask]


def random_mask(n):
    return np.random.binomial(1, 0.5, n).astype(bool)


def linreg(M, v):
    X_train, X_test, y_train, y_test = train_test_split(M, v, test_size=0.2, random_state=0)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_ = lr.predict(X_test)
    return mean_squared_error(y_test, y_)

# Populations

We define the individual object. When an individual is created their genome is generated randomly up to the number of genes needed, but the fitness is not calculated to avoid unnecesary computations. The fitness is only computed (and stored) when comparing individuals.

We have also implemented multiple sources of variability. We have punctual mutations (change individual bits, or between any bases out of a set if there is more than 2 options) and transpositions (one substring gets moved to a different position) for one individual, and crossovers between 2 individuals to give 2 new individuals. Transpositions are not used since they are too destructive.

In [5]:
class Pindividual:
    def __init__(self, chromosome):
        length = len(list(chromosome))
        if length > GENES:
            # print("WARNING: truncating chromosome")
            ch = list(chromosome)[:GENES]
        elif length < GENES:
            # print("WARNING: chromosome too short")
            ch = list(chromosome) + [random.randint(0, 1) for i in range(GENES - length)]
        else:
            ch = list(chromosome)
        self.chromosome = [bool(i) for i in ch]
        self.fitness = 0

    def __str__(self):
        # return f"fitness: {self.calculate_fitness()}"
        string = ""
        for i in self.chromosome:
            if i:
                string = string + "1"
            else:
                string = string + "0"
        return f"{string}, fitness: {round(self.calculate_fitness())}"

    def copy(self):
        return Pindividual(self.chromosome)

    def calculate_fitness(self):
        self.fitness = linreg(mask_X(X, self.chromosome), y)
        return self.fitness

    def __lt__(self, other):
        if self.fitness == 0:
            self.calculate_fitness()
        if other.fitness == 0:
            other.calculate_fitness()

        return self.fitness < other.fitness

    def crossover(self, partner):
        midpoint = random.randint(0, len(self.chromosome))
        child1 = self.chromosome[:midpoint] + partner.chromosome[midpoint:]
        child2 = self.chromosome[midpoint:] + partner.chromosome[:midpoint]
        return Pindividual(child1), Pindividual(child2)

    def punctual_mutation(self, prob):
        for i in range(len(self.chromosome)):
            if random.random() < prob:
                # mutable_bases = bases[:]
                # mutable_bases.remove(self.chromosome[i])
                # self.chromosome[i] = random.choice(mutable_bases)
                self.chromosome[i] = not self.chromosome[i]
        self.fitness = 0

    def transposition_mutation(self, prob):
        if random.random() > prob:
            return
        MAXSIZE = np.floor(GENES / 4)
        size = random.randint(1, MAXSIZE)
        place1 = random.randint(0, GENES - size - 1)
        place2 = place1
        while (abs(place1 - place2) < size):
            place2 = random.randint(0, GENES - size - 1)
        if place1 > place2:
            aux = place2
            place2 = place1
            place1 = aux
        print(size, place1, place2)
        interposon1 = self.chromosome[place1:place1 + size]
        interposon2 = self.chromosome[place2:place2 + size]
        self.chromosome = self.chromosome[:place1] + interposon2 + self.chromosome[place1 + size:place2] + interposon1 + self.chromosome[place2 + size:]
        self.fitness = 0

Another important function we implemented is _evolve_. It runs a selection method on a population a given amount of times and records the best individual in each generation.

In [6]:
def evolve(popu, gen, selection, param, best):
    for i in range(gen):
        print(f"gen {i+1}, time: ", end='')
        t1 = time.time()
        popu = selection(popu, param)
        t2 = time.time()
        print(f"{round(t2-t1,5)}s")
        # best.append(min(popu))
    print(min(popu))
    return popu

# Selection Methods

All our methods take a population and 1 parameter, and apply some technique to generate the next generation of the population. We have designed them as such to control everything from the _evolve_ function.

Direct selection takes the _n_ fittest individuals of a population and saves them (as in Noah's ark) to then repopulate from them into a whole new population. The repopulation is done first with crossovers between individuals until we reach a given threshold, and then mutating the saved individuals (fittest from previous generation and new ones) until we reach the wanted population size.

It usually results in the best outcomes, although it's slower since all individuals are evaluated.

In [7]:
def direct_selection(population, ark_capacity):
    # take best
    saved = []
    for i in population:
        if len(saved) < ark_capacity:
            bisect.insort(saved, i)
            continue
        if i < saved[-1]:
            bisect.insort(saved, i)
        if len(saved) > ark_capacity:
            saved.pop()

    # keep saved
    new_popu = []
    for i in saved:
        elem = i.copy()
        new_popu.append(elem)

    # aber cojan
    crossings = int((POPULATION_SIZE - ark_capacity) * CROSSOVER_RATE / 2)
    for i in range(crossings):
        # from OG (exploitation)
        # parent1 = random.randint(0, ark_capacity - 1)
        # parent2 = random.randint(0, ark_capacity - 1)
        # child1, child2 = saved[parent1].crossover(saved[parent2])
        # from new (exploration)
        parent1 = random.randint(0, len(new_popu) - 1)
        parent2 = random.randint(0, len(new_popu) - 1)
        child1, child2 = new_popu[parent1].crossover(new_popu[parent2])

        new_popu.append(child1)
        new_popu.append(child2)

    # fill with mutants
    while len(new_popu) < POPULATION_SIZE:
        pos = random.randint(0, len(new_popu) - 1)
        elem = new_popu[pos].copy()
        elem.punctual_mutation(MUTATION_RATE)
        new_popu.append(elem)

    return new_popu

Tournament selection takes a sample of the population and selects the fittest. Then, among the fittest ones we crossover until we get a new population. Some mutations are done before inserting in next generation.

Since crossover is expensive, we only do it in proportion the fixed _CROSSOVER\_RATE_.

This method is faster since we do less evaluations on average, but usually doesn't reach the same results as direct selection

In [8]:
def tournament(population, selection_size):
    new_popu = []
    while len(new_popu) < POPULATION_SIZE:
        parent1 = min(random.sample(population, selection_size))
        parent2 = min(random.sample(population, selection_size))
        if (random.random() < CROSSOVER_RATE):
            ch1, ch2 = parent1.crossover(parent2)
            ch1.punctual_mutation(MUTATION_RATE)
            ch2.punctual_mutation(MUTATION_RATE)
            new_popu.append(ch1)
            new_popu.append(ch2)
        else:
            parent1.punctual_mutation(MUTATION_RATE)
            parent1.fitness = 0
            parent2.punctual_mutation(MUTATION_RATE)
            parent2.fitness = 0
            new_popu.append(parent1)
            new_popu.append(parent2)
    return new_popu

# Analysis

Create 2 identical populations and use a different selection in each.

Since we don't want to give any of them a time advantage, we don't evaluate the populations to get the fittest at the cost of not having plots for fitness (for now).

In [9]:
random.seed(SEED)
popu1 = [Pindividual([]) for _ in range(POPULATION_SIZE)]
popu2 = []
for i in popu1:
    popu2.append(i.copy())

t1 = time.time()
evolved = evolve(popu1, GENERATIONS, tournament, 10,[])
t2 = time.time()
print(f"Direct selection total time: {t2-t1}s")

t1 = time.time()
evolved = evolve(popu2, GENERATIONS, direct_selection, int(POPULATION_SIZE / 10),[])
t2 = time.time()
print(f"Direct selection total time: {t2-t1}s")

gen 0, time: 5.87915s
gen 1, time: 4.00186s
gen 2, time: 5.04862s
gen 3, time: 4.69017s
gen 4, time: 5.86879s
gen 5, time: 6.14513s
gen 6, time: 4.74571s
gen 7, time: 6.13961s
gen 8, time: 4.58077s
gen 9, time: 6.05976s
gen 10, time: 7.24779s
gen 11, time: 6.71922s
gen 12, time: 6.9208s
gen 13, time: 6.68885s
gen 14, time: 5.52632s
gen 15, time: 5.12792s
gen 16, time: 6.3786s
gen 17, time: 5.83658s
gen 18, time: 4.76682s
gen 19, time: 6.50933s
1011110011001010001010110111100101100101011, fitness: 4637022674
Direct selection total time: 119.1733615398407s
gen 0, time: 4.26063s
gen 1, time: 6.0336s
gen 2, time: 6.26672s
gen 3, time: 6.43974s
gen 4, time: 5.53676s
gen 5, time: 5.73761s
gen 6, time: 6.1825s
gen 7, time: 5.84574s
gen 8, time: 7.03976s
gen 9, time: 7.15841s
gen 10, time: 6.81911s
gen 11, time: 6.64393s
gen 12, time: 5.65876s
gen 13, time: 6.9498s
gen 14, time: 5.70735s
gen 15, time: 4.96238s
gen 16, time: 5.70612s
gen 17, time: 5.90016s
gen 18, time: 6.95658s
gen 19, time: 4